In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

#경로 설정
%cd /content/drive/My Drive/llm_to_diffusion

In [1]:
%pip install diffusers
%pip install -q -U google-generativeai
%pip install accelerate

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gcn-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gcn-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATIO

In [2]:
import os
import sys
import random
from io import BytesIO

import cv2
import numpy as np
import h5py
import matplotlib.pyplot as plt
from PIL import Image
import requests

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.models.vgg as vgg

from diffusers import StableDiffusionInpaintPipeline
from huggingface_hub import hf_hub_download

from read_dataset import read_dataset
from model import segmentation_model
from train import train
from segmentation import segementation
from diffusion import *
from llm import *
from preprop import *

# Jupyter Notebook 내에서 matplotlib 출력 설정
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 32

model = segmentation_model().to(device)

# 하이퍼파라이터
epoch = 1
learning_rate = 0.00001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer.zero_grad()
criterion = nn.CrossEntropyLoss()

# **1. Train**

In [ ]:
file_path = 'dataset/G2_train.h5'
data_len = 10000
img_list, gt_list = read_dataset(file_path, data_len)

In [ ]:
train(img_list, gt_list, model, epoch, learning_rate, optimizer, criterion, data_len)

# **2. Inference**

## **(1) Set the Model into Eval Mode**

In [ ]:
model = segmentation_model()

# 가중치를 CPU로 로드
model.load_state_dict(torch.load('/content/drive/MyDrive/OUTTA 딥러닝 컨텐츠/final/model_state_dict4.pth', map_location=torch.device('cpu')))

# 모델을 평가 모드로 설정
model.eval()

## **(2) Load Image**

In [ ]:
image = np.load('./test_image3.npy')
plt.imshow(image)
plt.show()

# 결과 시각화
vis_output_model = segementation(image, model)
plt.imshow(vis_output_model)
plt.show()

## **(3) Inference**

### **1) LLM**

In [ ]:
prompt = """A Red short skirt, high quality, detailed, cute, with a lot of details,detailed, cyberpunk,
futuristic, with a lot of details, and a lot of colors."""
negative_prompt = "ugly"

text = LLM(prompt)
text.text

### **2) Segmentation & Inpainting**
- SAM for Segmentation

- Stable Diffusion for inpainting

In [ ]:
sd_pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
).to(device)

In [ ]:
image_source_pil, image_mask_pil = preprop_for_diffusion(image, vis_output_model)

In [ ]:
generated_image = generate_image(image=image_source_pil, mask=image_mask_pil, prompt=prompt, negative_prompt=negative_prompt, pipe=sd_pipe, seed=seed, device=device)
resized_image = generated_image.resize((generated_image.width * 3, generated_image.height * 3))
resized_image